# Model generation and training


This code aims to create the VAE model with the hyperparameters we chose before. It trains it on the entire dataset and saves it into h5 files. Once this model is generated, the encoder, generator and VAE can be imported and used in other files.

The hyperparameters chosen are :
- batchsize = 1
- intermediate dimension = 64
- latent dimension = 2
- number of epochs = 120

This code was used to generate and train the final model based on the real data, that is saved as ..._final.h5 (can be found in src/ml/samplers/weights_files) and used in the final project to generate the synthetic data.

This code can be run again, but will use the fake data for confidentiality.

In [1]:
#Importations
from keras import backend as K
from keras import losses
from keras.layers import Input, LSTM, Dense, Lambda, TimeDistributed
from keras.models import Model
import pickle
from src.ml.samplers.final_utils import *
from tensorflow.python.framework.ops import disable_eager_execution
import numpy as np
import matplotlib.pyplot as plt
disable_eager_execution()
from keras.models import load_model

In [2]:
initial_data, max_nb_clicks, num_encoder_clicks, input_clicks, inverse_clicks_dict, click_dict, input_data, decoder_input_data = get_text_data()

Number of samples: 254
Number of unique input click types: 22
Max number of clicks for one student: 819


In [3]:
#Data splitting
one_hot_train = input_data[:220,:] 
decoder_input_train = decoder_input_data[:220,:]
one_hot_validation = input_data[220:,:] 
decoder_input_validation = decoder_input_data[220:,:]

In [5]:
#Parameters
input_dim=len(input_data[0][0]) #corresponds to the length of the one-hot vectors
batch_size = 1
intermediate_dim = 64
latent_dim = 2
num_epochs = 120

#Model creation
vae, encoder, generator, stepper = create_lstm_vae(input_dim, batch_size, intermediate_dim, latent_dim)

#Model training
vae.fit(x=[input_data, decoder_input_data], y=input_data,batch_size=batch_size, epochs=num_epochs, verbose=1, validation_data = ([one_hot_validation,decoder_input_validation], one_hot_validation))

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, None, 23)]           0         []                            
                                                                                                  
 lstm_2 (LSTM)               (None, 64)                   22528     ['input_6[0][0]']             
                                                                                                  
 dense_4 (Dense)             (None, 2)                    130       ['lstm_2[0][0]']              
                                                                                                  
 dense_5 (Dense)             (None, 2)                    130       ['lstm_2[0][0]']              
                                                                                            

2023-12-15 13:34:23.838730: W tensorflow/c/c_api.cc:305] Operation '{name:'training_2/Adam/beta_2/Assign' id:3099 op device:{requested: '', assigned: ''} def:{{{node training_2/Adam/beta_2/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/Adam/beta_2, training_2/Adam/beta_2/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


254/254 [==============================] - ETA: 0s - loss: 0.2903

/Users/valentinedelevaux/anaconda3/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-12-15 13:34:47.199150: W tensorflow/c/c_api.cc:305] Operation '{name:'loss_1/mul' id:2761 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/time_distributed_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


254/254 [==============================] - 24s 94ms/sample - loss: 0.2903 - val_loss: 0.2017
Epoch 2/120
254/254 [==============================] - 24s 95ms/sample - loss: 0.2112 - val_loss: 0.1701
Epoch 3/120
254/254 [==============================] - 24s 95ms/sample - loss: 0.1869 - val_loss: 0.1542
Epoch 4/120
254/254 [==============================] - 24s 95ms/sample - loss: 0.1744 - val_loss: 0.1446
Epoch 5/120
254/254 [==============================] - 24s 96ms/sample - loss: 0.1636 - val_loss: 0.1388
Epoch 6/120
254/254 [==============================] - 25s 97ms/sample - loss: 0.1569 - val_loss: 0.1315
Epoch 7/120
254/254 [==============================] - 24s 96ms/sample - loss: 0.1512 - val_loss: 0.1275
Epoch 8/120
254/254 [==============================] - 24s 95ms/sample - loss: 0.1471 - val_loss: 0.1245
Epoch 9/120
254/254 [==============================] - 24s 95ms/sample - loss: 0.1445 - val_loss: 0.1222
Epoch 10/120
254/254 [==============================] - 24s 95ms/sa

In [6]:
#Model saving
vae.save('model_vae_.h5')
generator.save_weights('generator_weights_.h5')
encoder.save_weights('encoder_weights_.h5')